
# Tarea

**7p**: Implementar un algoritmo MAP REDUCE que extraiga el tamaño de las palabras y que palabras tienen dicho tamaño de todas las palabras del texto.

ej:
Para el texto:  “En un lugar de la mancha”
El resultado seria: {2: [“el”,”un”,”de”,”la”],5: [“lugar”],6: [mancha”]}

**2p**: Incluir limpieza de caracteres que no sean palabras persé como puntuaciones, exclamaciones , saltos de línea, etc.

**1p**: Añadir typing a las funciones map y reduce

# Condiciones

El algoritmo usado para implementar tiene que ser map reduce.

El fichero tiene que tener el nombre del alumno con _ y en formato .pynb ej lucas_alvarez_arguero.pynb

El notebook se debe de poder ejecutar de principio a fin sin fallar.

Usar las funciones que ofrece Python de manera nativa, nada de usar frameworks como pyspark etc.

In [29]:
!pip install wget

In [30]:
import random
import math
from typing import *
import re


In [31]:
from functools import reduce

In [32]:
import wget

In [33]:
url = 'https://gist.github.com/jsdario/6d6c69398cb0c73111e49f1218960f79/raw/8d4fc4548d437e2a7203a5aeeace5477f598827d/el_quijote.txt'


In [34]:
file_path=wget.download(url)

In [35]:
file_path

'el_quijote (1).txt'

In [36]:
file_1= open(file_path,'r')

In [37]:
type(file_1)

_io.TextIOWrapper

In [38]:
lines= file_1.readlines()

In [39]:
lines[:6]

['DON QUIJOTE DE LA MANCHA\n',
 'Miguel de Cervantes Saavedra\n',
 '\n',
 'PRIMERA PARTE\n',
 'CAPÍTULO 1: Que trata de la condición y ejercicio del famoso hidalgo D. Quijote de la Mancha\n',
 'En un lugar de la Mancha, de cuyo nombre no quiero acordarme, no ha mucho tiempo que vivía un hidalgo de los de lanza en astillero, adarga antigua, rocín flaco y galgo corredor. Una olla de algo más vaca que carnero, salpicón las más noches, duelos y quebrantos los sábados, lentejas los viernes, algún palomino de añadidura los domingos, consumían las tres partes de su hacienda. El resto della concluían sayo de velarte, calzas de velludo para las fiestas con sus pantuflos de lo mismo, los días de entre semana se honraba con su vellori de lo más fino. Tenía en su casa una ama que pasaba de los cuarenta, y una sobrina que no llegaba a los veinte, y un mozo de campo y plaza, que así ensillaba el rocín como tomaba la podadera. Frisaba la edad de nuestro hidalgo con los cincuenta año

In [40]:
def word_lengths(row: str) -> Dict[int, Set[str]]:
    row_clean = re.sub(r'[^a-záéóúííñ\s]', ' ', row.lower())
    num_letters = {}
    for word in row_clean.split():
        letters = len(word)
        if letters in num_letters.keys():
            num_letters[letters].add(word)
        else:
            num_letters[letters] = {word}
    return num_letters


In [41]:
entrada_1=word_lengths("hola mundo m-AAdo ")
entrada_1

{4: {'aado', 'hola'}, 5: {'mundo'}, 1: {'m'}}

In [42]:
entrada_2=word_lengths(lines[4])
entrada_2

{9: {'capítulo', 'ejercicio'},
 3: {'del', 'que'},
 5: {'trata'},
 2: {'de', 'la'},
 10: {'condición'},
 1: {'d', 'y'},
 6: {'famoso', 'mancha'},
 7: {'hidalgo', 'quijote'}}

In [43]:
map_result=list(map(lambda row: word_lengths(row),lines))

In [44]:
'Í'.lower()

'í'

In [45]:
map_result[:2]

[{3: {'don'}, 7: {'quijote'}, 2: {'de', 'la'}, 6: {'mancha'}},
 {6: {'miguel'}, 2: {'de'}, 9: {'cervantes'}, 8: {'saavedra'}}]

In [46]:
def reduce_word_lengths(num_letters_a: Dict[int, Set[str]], num_letters_b: Dict[int, Set[str]]) -> Dict[int, List[str]]:
    num_letters = num_letters_a.copy()
    for n in num_letters_b.keys():
        if n in num_letters.keys():
            num_letters[n].update(num_letters_b[n])
        else:
            num_letters[n] = num_letters_b[n]
    return num_letters

In [47]:
reduce_word_lengths(entrada_2, entrada_1)

{9: {'capítulo', 'ejercicio'},
 3: {'del', 'que'},
 5: {'mundo', 'trata'},
 2: {'de', 'la'},
 10: {'condición'},
 1: {'d', 'm', 'y'},
 6: {'famoso', 'mancha'},
 7: {'hidalgo', 'quijote'},
 4: {'aado', 'hola'}}

In [48]:
reduced = reduce(lambda row_a, row_b: reduce_word_lengths(row_a, row_b), map_result)
reduced = {n: list(s) for (n, s) in reduced.items()}